In [2]:
from PIL import Image
import numpy as np
from DeepLab_V3 import Deeplab_v3
from Datasource import test_dataloader,train_dataloader
import torch
import torch.nn as nn
model = Deeplab_v3()
model.load_state_dict(torch.load('model_weights_12.pth'))
model.eval() #模型评估
model = model.cuda()
def show_result(dataloader, model):
    count_of_img=1
    with torch.no_grad():
        for X, y in dataloader:
            X = X.cuda()
            pred = model(X)
            pred = pred.cpu()
            pred = nn.Softmax(dim=1)(pred)#要归一化一下，这样才能形成概率，之前网络结构中没有这一步是因为交叉熵函数要输入的是raw data
            #pred = pred.int()
            pred = pred.argmax(dim=1)
            pred = pred.int()
            pred = np.array(pred)
            y = y.int()#直接转换会有问题，因为当初我们取label时为了使用交叉熵函数转成过Long型
            y = np.array(y)
            batch_size = pred.shape[0]
            for i in range(batch_size):
                tensor_this_time = pred[i]
                result_this_time = Image.fromarray(tensor_this_time)
                result_this_time = result_this_time.convert("L")
                result_this_time.putpalette([0, 0, 0, 145, 24, 147, 23, 145, 147, 145, 147, 24, 180, 70, 70, \
                                             70, 180, 70, 70, 70, 180, 100, 200, 150, 100, 150, 200, 150, 100, \
                                             200, 150, 200, 100, 200, 100, 150, 200, 150, 100, 50, 110, 220, 50, \
                                             220,110, 110, 50, 220, 110, 220, 50, 220, 50, 110, 220, 110, 50, \
                                             170, 80, 90, 255, 255, 255])
                result_this_time.save("result_"+str(count_of_img)+".png")
                label_this_time = y[i]
                #label_this_time = label_this_time.int()
                label_this_time = Image.fromarray(label_this_time)
                label_this_time = label_this_time.convert("L")
                label_this_time.putpalette([0, 0, 0, 145, 24, 147, 23, 145, 147, 145, 147, 24, 180, 70, 70, \
                                             70, 180, 70, 70, 70, 180, 100, 200, 150, 100, 150, 200, 150, 100, \
                                             200, 150, 200, 100, 200, 100, 150, 200, 150, 100, 50, 110, 220, 50, \
                                             220,110, 110, 50, 220, 110, 220, 50, 220, 50, 110, 220, 110, 50, \
                                             170, 80, 90, 255, 255, 255])
                label_this_time.save("label_"+str(count_of_img)+".png")
                count_of_img+=1

show_result(test_dataloader,model)